In [5]:
import requests
import json
import pandas as pd
from pandas import json_normalize
import config
from db_manager import DBManager
import os
from rest_api import Api
from datetime import date
from dateutil.relativedelta import relativedelta

#### Show all columns without truncating

In [2]:
pd.set_option('display.max_columns', None)

Today's date

In [4]:
today = date.today()
print(today)

2023-08-16


Adding months to today's date. Near term travel dates 30 - 60 days ahead

In [15]:
departure_date_1_month = today + relativedelta(months=1)
departure_date_2_month = today + relativedelta(months=2)
arrival_date_1_month = departure_date_2_month
arrival_date_2_month = today + relativedelta(months=3)
print(f'Departure_dates:{departure_date_1_month} and {departure_date_2_month} \nArrival dates:{arrival_date_1_month} and {arrival_date_2_month}')

Departure_dates:2023-09-16 and 2023-10-16 
Arrival dates:2023-10-16 and 2023-11-16


#### Travel payouts API to get flight search results

In [3]:
api = Api()
params=config.params
response = api.make_flight_api_request(**params)
print(response)

https://api.flightapi.io/roundtrip/64d8ff40c4386061ed958548/BER/MAA/2023-09-15/2023-10-15/1/0/0/Economy/EUR
API response: {'legs': [{'id': 'MAA-BER:KL3689~15:KL878~16:KL1823~16:1', 'departureTime': '22:10', 'arrivalTime': '11:30', 'duration': '16h 50m', 'departureAirportCode': 'MAA', 'arrivalAirportCode': 'BER', 'airlineCodes': ['KL'], 'stopoverAirportCodes': ['BOM', 'AMS'], 'allianceCodes': ['sky_team'], 'stopoversCount': 2, 'departureTimeMinutes': 1330, 'arrivalTimeMinutes': 690, 'departureDateTime': '2023-10-15T22:10:00.000+05:30', 'arrivalDateTime': '2023-10-16T11:30:00.000+02:00', 'stopoverDurationMinutes': 260, 'durationMinutes': 1010, 'overnight': True, 'stopoverDuration': '04h 20m', 'durationDays': 1, 'longStopover': True, 'segments': [{'durationMinutes': 115, 'stopoverDurationMinutes': 140, 'departureAirportCode': 'MAA', 'arrivalAirportCode': 'BOM', 'airlineCode': 'KL', 'cabin': 'economy', 'designatorCode': 'KL3689', 'departureDateTime': '2023-10-15T22:10:00.000+05:30', 'arriv

#### df_legs is the main table that contains leg id, dep time, arr time, dep airport, arr airport, stopovers, duration

In [ ]:
df_legs = json_normalize(dict['legs']) 

KeyError: 'legs'

Dropping unwanted columns

In [ ]:
df_legs=df_legs.drop(columns=['departureTimeMinutes','arrivalTimeMinutes','segments','operatingAirlineCodes','stopoverCode','shortStopover','earlyDeparture','lateArrival','newAircraft','oldAircraft','highlyRatedCarrier','score'])

In [ ]:
df_legs.head()

,id,departureTime,arrivalTime,duration,departureAirportCode,arrivalAirportCode,airlineCodes,stopoverAirportCodes,allianceCodes,stopoversCount,departureDateTime,arrivalDateTime,stopoverDurationMinutes,durationMinutes,overnight,stopoverDuration,durationDays,longStopover
0,MAA-BER:KL2092~15:KL1228~15:KL1825~15:1,01:20,13:45,15h 55m,MAA,BER,[KL],"[CDG, AMS]",[sky_team],2,2023-10-15T01:20:00.000+05:30,2023-10-15T13:45:00.000+02:00,160,955,False,02h 40m,0,False
1,BER-MAA:AF1535~15:AF108~16:0,20:30,23:30,23h 30m,BER,MAA,[AF],[CDG],[sky_team],1,2023-09-15T20:30:00.000+02:00,2023-09-16T23:30:00.000+05:30,695,1410,True,11h 35m,1,True
2,MAA-BER:KL3689~15:KL878~16:KL1823~16:1,22:10,11:30,16h 50m,MAA,BER,[KL],"[BOM, AMS]",[sky_team],2,2023-10-15T22:10:00.000+05:30,2023-10-16T11:30:00.000+02:00,260,1010,True,04h 20m,1,True
3,BER-MAA:AF1135~15:AF212~15:AF3326~16:0,06:00,07:30,22h,BER,MAA,[AF],"[CDG, BOM]",[sky_team],2,2023-09-15T06:00:00.000+02:00,2023-09-16T07:30:00.000+05:30,565,1320,True,09h 25m,1,True
4,BER-MAA:AF1235~15:AF108~16:0,15:20,23:30,28h 40m,BER,MAA,[AF],[CDG],[sky_team],1,2023-09-15T15:20:00.000+02:00,2023-09-16T23:30:00.000+05:30,1005,1720,True,16h 45m,1,True


#### df_trips is the df that acts as an intermediate df between legs and fares array. It contains the trip id and the legs id

In [ ]:
df_trips = json_normalize(dict['trips'])

Exploding the legIds column to spread the list of ledIds to several rows

In [ ]:
df_trips=df_trips.explode('legIds')

In [ ]:
df_trips.head()

,id,code,legIds
0,5b54aaa96228127cmsr:KL1820~15-KL871~15-KL3714~...,KL1820~15-KL871~15-KL3714~16=KL3689~15-KL878~1...,BER-MAA:KL1820~15:KL871~15:KL3714~16:0
0,5b54aaa96228127cmsr:KL1820~15-KL871~15-KL3714~...,KL1820~15-KL871~15-KL3714~16=KL3689~15-KL878~1...,MAA-BER:KL3689~15:KL878~16:KL1823~16:1
1,5b54aaa96228127cmsr:AF1135~15-AF108~15=AF107~1...,AF1135~15-AF108~15=AF107~15-AF1834~15,BER-MAA:AF1135~15:AF108~15:0
1,5b54aaa96228127cmsr:AF1135~15-AF108~15=AF107~1...,AF1135~15-AF108~15=AF107~15-AF1834~15,MAA-BER:AF107~15:AF1834~15:1
2,5b54aaa96228127cmsr:AF1235~15-AF108~16=AF107~1...,AF1235~15-AF108~16=AF107~15-AF1534~15,BER-MAA:AF1235~15:AF108~16:0


#### df_fares is where we have the price and currency info for the trips seen in legs array. Here you have columhns such as tripId, currencyCode and totalAmount

In [ ]:
df_fares=json_normalize(dict['fares'])

Rename columns that follows a certain pattern

In [ ]:
df_fares.columns = df_fares.columns.str.replace("^price.", "", regex=True)


In [ ]:
df_fares=df_fares.drop(columns=['paymentFees'])

In [ ]:
df_fares.head()

,id,providerCode,handoffUrl,ecpc,remainingSeatsCount,conditionIds,legConditionIds,refundable,exchangeable,tags,score,tripId,totalAmount,totalAmountUsd,amount,amountUsd,originalAmount,originalAmountUsd,amountPerAdult,amountPerChild,amountPerInfant,taxAmount,taxAmountUsd,totalTaxAmount,totalTaxAmountUsd,currencyCode,paymentFeeAmountUsd,bookingFee,bookingFeeUsd,totalBookingFee,totalBookingFeeUsd
0,5b54aaa96228127cmsr:airfrance.com:c884e5b9b331abc,airfrance.com,https://handoff.wego.com/flights/continue?curr...,0.126223,0,[],[],False,False,[],865.44165,5b54aaa96228127cmsr:AF1135~15-AF108~15=AF107~1...,704,771.2283,704,771.2283,703.90,771.2283,703.90,0,0,0,0,0,0,EUR,0,0,0,0,0
1,5b54aaa96228127cmsr:airfrance.com:d61adf3e3522...,airfrance.com,https://handoff.wego.com/flights/continue?curr...,0.126223,0,[],[],False,False,[],751.61170,5b54aaa96228127cmsr:AF1535~15-AF108~16=AF107~1...,704,771.2283,704,771.2283,703.90,771.2283,703.90,0,0,0,0,0,0,EUR,0,0,0,0,0
2,5b54aaa96228127cmsr:airfrance.com:2313df268aac...,airfrance.com,https://handoff.wego.com/flights/continue?curr...,0.126223,0,[],[],False,False,[],773.63770,5b54aaa96228127cmsr:AF1135~15-AF108~15=AF107~1...,710,778.0322,710,778.0322,710.11,778.0322,710.11,0,0,0,0,0,0,EUR,0,0,0,0,0
3,5b54aaa96228127cmsr:airfrance.com:b0147680c5c5...,airfrance.com,https://handoff.wego.com/flights/continue?curr...,0.126223,0,[],[],False,False,[],659.80774,5b54aaa96228127cmsr:AF1535~15-AF108~16=AF107~1...,710,778.0322,710,778.0322,710.11,778.0322,710.11,0,0,0,0,0,0,EUR,0,0,0,0,0
4,5b54aaa96228127cmsr:airfrance.com:53e9eb430f58...,airfrance.com,https://handoff.wego.com/flights/continue?curr...,0.126223,0,[],[],False,False,[],708.07280,5b54aaa96228127cmsr:AF1835~15-AF108~16=AF107~1...,714,782.0971,714,782.0971,713.82,782.0971,713.82,0,0,0,0,0,0,EUR,0,0,0,0,0


In [ ]:
df_fares=df_legs.drop(columns=['departureTimeMinutes','arrivalTimeMinutes','segments','operatingAirlineCodes','stopoverCode','shortStopover','earlyDeparture','lateArrival','newAircraft','oldAircraft','highlyRatedCarrier','score'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   89 non-null     object 
 1   providerCode         89 non-null     object 
 2   handoffUrl           89 non-null     object 
 3   ecpc                 89 non-null     float64
 4   remainingSeatsCount  89 non-null     int64  
 5   conditionIds         89 non-null     object 
 6   legConditionIds      89 non-null     object 
 7   refundable           89 non-null     bool   
 8   exchangeable         89 non-null     bool   
 9   tags                 89 non-null     object 
 10  score                89 non-null     float64
 11  tripId               89 non-null     object 
 12  totalAmount          89 non-null     int64  
 13  totalAmountUsd       89 non-null     float64
 14  amount               89 non-null     int64  
 15  amountUsd            89 non-null     float

In [ ]:
password=config.db_password
escaped_password = quote_plus(password) 
port=config.db_port
database=config.database
host=config.db_host

AttributeError: 'ConfigParser' object has no attribute 'db_password'

In [ ]:
print(escaped_password)

Keerthana%40123


Establish connection With postgresql DB. Note to use config file for storing passwords separately from the python code

In [ ]:
engine = create_engine(f'postgresql://postgres:{escaped_password}@{host}:{port}/{database}')

In [ ]:
df_legs.to_sql('flight_details', engine, if_exists='append', index=False)

In [ ]:
df_trips.to_sql('trips',engine, if_exists='append', index=False)

In [ ]:
df_fares.to_sql('fares',engine,if_exists='append',index=False)